In [78]:
import pandas as pd
import numpy as np

Import dataset

In [2]:
df = pd.read_csv("weekly_gai_sector.csv")

Sort the values out - by Sector ID, Year and Week

In [3]:
df= df.sort_values(["Sectorid","Year","Week"])

Get unique set of SectorID

In [4]:
unique_sector = df["Sectorid"].unique()

Add a date column

In [5]:
df["Date"]=df["Year"].astype(str) + "-" +df["Week"].astype(str)

There are a total of 1227 sectors

In [6]:
len(unique_sector)

1227

Choose the sector

In [8]:
sector_chosen = unique_sector[1000]
df_sector = df[df["Sectorid"]==sector_chosen].reset_index(drop=True)

In [9]:
df_sector.head()

,Year,Sectorid,GAI,Week,threshold_0.05,threshold_0.1,threshold_0.2,Date
0,2018,FL648,0.131579,1,1,1,0,2018-1
1,2018,FL648,0.071429,2,1,0,0,2018-2
2,2018,FL648,0.210526,3,1,1,1,2018-3
3,2018,FL648,0.481481,4,1,1,1,2018-4
4,2018,FL648,0.131579,5,1,1,0,2018-5


Write a function to count consecutive lengths

In [94]:
def count_length(x):
    container = []
    i=0
    counter= 0
    while i <len(x):
       # print(i)
        if x[i]==1:
            counter+=1
            if i==len(x)-1:
                container.append([i+1,counter])
            i+=1
        elif x[i]==0:
           # print(i)
            if  i!=0 and x[i-1]==1:
                container.append([i,counter])
            counter=0
            i+=1
            continue
    return container

In [95]:
foo=df[df["Sectorid"]=="CO1"]["threshold_0.05"].reset_index(drop=True)

In [96]:
np.array(foo)

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0,
       0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0],
      dtype=int64)

In [97]:
count_length(foo)

[]

Write function to return the dates

In [106]:
def return_dates(container,masterframe,threshold):
    filtered_container =  list(filter(lambda x: x[1]>=threshold, container)) 
  #  print(filtered_container)
    lengths = list(map(lambda x: x[1],filtered_container))
    date_index = [[x-y,x] for [x,y] in filtered_container]
    #print("="*30)
    #print(date_index)
    #build frame
    frame = [[masterframe["Date"][x],masterframe["Date"][y-1]] for [x,y] in date_index]
    frame = pd.DataFrame(frame,columns = ["Start","End"])
    frame["Duration"]=lengths
  #  print(frame)
   # print(lengths)
    '''
        results = []
    for i in range(0,len(frame)):
        temp = pd.DataFrame(frame[i]).transpose()
       # print(temp)
        temp["Length"]=lengths[i]
        results.append(temp)
    '''

    return frame



In [27]:
def return_dates(container,masterframe,threshold):
    filtered_container =  list(filter(lambda x: x[1]>=threshold, container)) 
  #  print(filtered_container)
    lengths = list(map(lambda x: x[1],filtered_container))
    date_index = [[x-y,x] for [x,y] in filtered_container]
    
    #build frame
    frame = [[masterframe["Date"][x:y]] for [x,y] in date_index]
  #  print(frame)
   # print(lengths)
    results = []
    for i in range(0,len(frame)):
        temp = pd.DataFrame(frame[i]).transpose()
       # print(temp)
        temp["Length"]=lengths[i]
        results.append(temp)
    return pd.concat(results)



In [98]:
bar=count_length(df_sector["threshold_0.05"])

In [75]:
foo = return_dates(bar,df_sector,4)

[[0, 17], [34, 38], [42, 72], [74, 81], [82, 95], [97, 169]]


In [76]:
foo.insert(0,"Sector","A")

In [77]:
foo

,Sector,Start,End,Duration
0,A,2018-1,2018-17,17
1,A,2018-35,2018-38,4
2,A,2018-43,2019-20,30
3,A,2019-23,2019-29,7
4,A,2019-31,2019-43,13
5,A,2019-46,2021-13,72


### Put everything together

In [130]:
def gen_cutoff(cutoff,min_threshold):
    container= {}
    for i,sector in enumerate(unique_sector):
      #  print("="*30)
       # print(f"{i}_{sector}")
        sector_chosen = unique_sector[i]
        df_sector = df[df["Sectorid"]==sector_chosen].reset_index(drop=True)
       # print(df_sector)
        #count consecutive lengths
        consec_length = count_length(df_sector[cutoff])
        #print("YAZ")
        if len(consec_length)>0:
            dateframe = return_dates(consec_length,df_sector,min_threshold)
            container[sector_chosen]=dateframe
            
    #filter out the values for sectors with at least one threshold met
    newDict = {key: value for (key, value) in container.items() if len(value) >0 }
    masterframe = pd.concat(newDict).reset_index().drop("level_1",axis=1).rename(columns={"level_0":"SectorID"})
    return masterframe
        
    

### Result Threshold 0.05

In [132]:
result_005 = gen_cutoff("threshold_0.05",4)

In [135]:
result_01 = gen_cutoff("threshold_0.1",4)

In [139]:
result_02 = gen_cutoff("threshold_0.2",4)

In [163]:
result_005.to_csv("result_005.csv",index=False)

In [164]:
result_01.to_csv("result_01.csv",index=False)

In [165]:
result_02.to_csv("result_02.csv",index=False)

In [137]:
result_005.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5394 entries, 0 to 5393
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   SectorID  5394 non-null   object
 1   Start     5394 non-null   object
 2   End       5394 non-null   object
 3   Duration  5394 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 168.7+ KB


In [136]:
result_01.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5259 entries, 0 to 5258
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   SectorID  5259 non-null   object
 1   Start     5259 non-null   object
 2   End       5259 non-null   object
 3   Duration  5259 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 164.5+ KB


In [140]:
result_02.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3891 entries, 0 to 3890
Data columns (total 4 columns):
 #   Column    Non-Null Count  Dtype 
---  ------    --------------  ----- 
 0   SectorID  3891 non-null   object
 1   Start     3891 non-null   object
 2   End       3891 non-null   object
 3   Duration  3891 non-null   int64 
dtypes: int64(1), object(3)
memory usage: 121.7+ KB


Filter out the results to include only sectors that have a value/ dataframe

In [116]:
newDict = {key: value for (key, value) in result.items() if len(value) >0 }

In [124]:
masterframe = pd.concat(newDict).reset_index().drop("level_1",axis=1).rename(columns={"level_0":"SectorID"})

In [125]:
masterframe.head()

,SectorID,Start,End,Duration
0,CO130,2021-6,2021-10,5
1,CO313,2020-6,2020-10,5
2,CO313,2020-12,2021-13,54
3,CO315,2020-32,2020-37,6
4,CO315,2020-39,2020-48,10


In [126]:
masterframe.to_csv("Sectors_Threshold.csv",index=False)